# ROC CURVES POST PROCESSING METHOD TO IMPROVE FAIRNESS (SEPARATION) IN LS-SVM


**Objectives**

We want to emphasize unfairness in some AI/ML algorithm and then try to improve fairness of the machine by using post-correction process. Particularly with ROC curves. 

Therefore, we generate 2 Classes $C_1$ et $C_2$. Each class has a 2 groups : one majority and one minority. Thus, the protected attribute is being in a minority.

The Goal is to show some misclassification of minority observations, and the correct it with ROC curves : the best operating point (the optimal pair of thresholds is hypothetically at the intersection of the 2 ROC curves. The next objective is to compute the threshold with the list of points we obtain by displaying both curves.

For this notebook, we work on **simulated data**.

## ROC CURVES

* abscissas : $P(\hat{Y} = 1 \mid Y = 0, A = a)$

* ordinates : $P(\hat{Y} = 1 \mid Y = 1, A =a)$

* Here, $A \in ZOO$. $A=0$ means "in majority" and $A=1$ means "in minority"

A point is plotted for a certain threshold $\ \xi$. The curve is the union of all points generated.

In [1]:
# import the modules we need
import numpy as np
%matplotlib qt
import matplotlib.pyplot as plt
import scipy.linalg
import scipy.special
import scipy.stats
from math import *
import random as r
pi = np.pi

In [2]:
cs = [ 8/16, 1/16, 5/16, 2/16 ] #ratio of each class
c2 = cs[-1] + cs[-2]
c1 = cs[0] + cs[1]
k = len(cs)
n = 1024 * 8
n_test = 1024 * 8
p = 1024 * 8

In [3]:
gamma = 1 # regularization
f = lambda t : np.exp(-t / 2) # for kernel method

# generate means
mu_c1 = (np.zeros((1, p))).T
mu_c1[0,:] = 2
mu_c2 = (np.zeros((1, p))).T
mu_c2[1, :] = 2

#print(mu_c1.shape)
#print(mu_c2)
beta = 0.65
cov = np.eye(p, dtype = float)
# generate covariances
#CM1 = (1/1000000) * np.eye(p, dtype = float) # majority
#Cm = np.diagflat([ 1 for _ in range(int(np.sqrt(p)))] + [0 for _ in range(p - int(np.sqrt(p)))]) # minority

#SM = scipy.linalg.sqrtm(CM)
#Sm = scipy.linalg.sqrtm(Cm)


# ROC A = 1
#print(mu2)
#print(mu1)

In [4]:
c10 = cs[0]
c20 = cs[2]
c11 = cs[1]
c21 = cs[3]

xip = c2 - c1
xi0 = c20 - c10
xi1 = c21 - c11

L = [xip, xi0, xi1]
L.sort()


precision = 750 * 3
hs = 500 * 2
 
xi_loop = list(np.linspace(-1, L[0] - L[0] / 10, hs)) + list(np.linspace(L[0] - L[0] / 10, L[0] + L[0] / 10, precision)) + list(np.linspace( L[0] + L[0] / 10, L[1] - L[1] / 10, hs)) + list(np.linspace( L[1] - L[1] / 10, L[1] + L[1] / 10, precision)) + list(np.linspace( L[1] + L[1] / 10, L[2] - L[2] / 2, hs)) + list(np.linspace( L[2] - L[2] / 2, L[2] + L[2] / 2, precision)) + list(np.linspace( L[2] + L[2] / 2, 1, hs))
xi_loop.sort()
nb_average_loop = 1

store_error = np.zeros((len(xi_loop), 8), dtype = float)

# targets are defined before the loop cause they don't move
# target Y
y = np.concatenate([-np.ones(int((cs[0] + cs[1]) * n)),np.ones(int((cs[2] + cs[3]) * n))])
        
# target Y_test
y_test = np.concatenate([-np.ones(int((cs[0] + cs[1]) * n_test)),np.ones(int((cs[2] + cs[3]) * n_test))])

X = np.zeros( (p,n) )
X_test = np.zeros( (p, n_test) )

# theory
def fprime(x):
    return - 1 / 2 * f(x)

def fseconde(x):
    return - 1 / 2 * fprime(x)

g_theory0 = np.zeros(n_test)
g_theory1 = np.zeros(n_test)

In [5]:
# THEORIC ROC A = 0
#ROC A = 0
# C1 : Y = 0, A = 0
# C2 : Y = 1, A = 1

tau = 2 * n / p
n0 = int((cs[0] + cs[2]) * n)

mu10 = mu_c1
mu20 = mu_c2

tau0 = 2 / p * np.trace(c1 * cov + c2 * cov)
D0 = - 2 * fprime(tau) / p * (np.linalg.norm(mu20 - mu10) ** 2)

E10 = c2 - c1 - 2 * c2 * c1 * c2 * gamma * D0
E20 = c2 - c1 + 2 * c1 * c1 * c2 * gamma * D0

V110 = 0
V120 = 0

V210 = 2 * (((fprime(tau))**2) / (p**2)) * np.linalg.norm(mu20 - mu10) ** 2
V220 = 2 * (((fprime(tau))**2) / (p**2)) * np.linalg.norm(mu20 - mu10) ** 2

V310 = 2 * ((fprime(tau)**2) / (n * (p**2))) * (np.trace(cov) / c1 + np.trace(cov) / c2)
V320 = 2 * ((fprime(tau)**2) / (n * (p**2))) * (np.trace(cov) / c1 + np.trace(cov) / c2)

VAR10 = float(8 * ((gamma * c2 * c1) ** 2) * (V310 + V210 + V110))
VAR20 = float(8 * ((gamma * c2 * c1) ** 2) * (V320 + V220 + V120))

In [6]:
# THEORIC ROC A = 1



n1 = int((cs[1] + cs[3]) * n)

mu11 = mu_c1 * beta
mu21 = mu_c2 * beta
# 2 * ( 1 - beta ** 2)  * np.trace(c1 * cov + c2 * cov) / p
tau1 = tau
D1 = - 2 * fprime(tau1) / p * (np.linalg.norm(mu21 - mu11) ** 2)

E11 = c2 - c1 - 2 * c2 * c1 * c2 * gamma * D1
E21 = c2 - c1 + 2 * c1 * c1 * c2 * gamma * D1

V111 = 0
V121 = 0

#V211 = 2 * (((fprime(tau1))**2) / (p**2)) * (np.linalg.norm(mu21 - mu11) ** 2) * (1 - beta ** 2)
#V221 = 2 * (((fprime(tau1))**2) / (p**2)) * (np.linalg.norm(mu21 - mu11) ** 2) * (1 - beta ** 2)

V211 = 2 * (((fprime(tau1))**2) / (p**2)) * (np.linalg.norm(mu21 - mu11) ** 2)
V221 = 2 * (((fprime(tau1))**2) / (p**2)) * (np.linalg.norm(mu21 - mu11) ** 2)

#V311 = 2 * ((fprime(tau1)**2) / (n * (p**2))) * (np.trace(cov) * ((1 - beta ** 2) ** 2) / c1 + np.trace(cov) * ((1 - beta ** 2) ** 2) / c2)
#V321 = 2 * ((fprime(tau1)**2) / (n * (p**2))) * (np.trace(cov) * ((1 - beta ** 2) ** 2) / c1 + np.trace(cov) * ((1 - beta ** 2) ** 2) / c2)


V311 = 2 * ((fprime(tau1)**2) / (n * (p**2))) * (np.trace(cov)  / c1 + np.trace(cov) / c2)
V321 = 2 * ((fprime(tau1)**2) / (n * (p**2))) * (np.trace(cov)  / c1 + np.trace(cov) / c2)

VAR11 = float(8 * ((gamma * c2 * c1) ** 2) * (V311 + V211 + V111))
VAR21 = float(8 * ((gamma * c2 * c1) ** 2) * (V321 + V221 + V121))

In [7]:
# GENERATE SCORES
seuil10 = 0
seuil11 = int(c10 * n_test)
seuil20 = int((c10 + c11) * n_test)
seuil21 = int(sum(cs[:-1]) * n_test)

g_theory = np.zeros(n_test)
for i in range(n_test):
    if i >= seuil10 and i < seuil11:
        g_theory[i] = r.gauss(E10, np.sqrt(VAR10))
    elif i >= seuil11 and i < seuil20:
        g_theory[i] = r.gauss(E11, np.sqrt(VAR11))
    elif i >= seuil20 and i < seuil21:
        g_theory[i] = r.gauss(E20, np.sqrt(VAR20))
    else:
        g_theory[i] = r.gauss(E21, np.sqrt(VAR21))


In [8]:
"""
for i in range(k):
    # class 1
    if i < 2:
        # majority
        if i == 0:
        #print(mu_c1.shape)
            X[: , int(np.sum(cs[:i])*n):int(np.sum(cs[:i+1])*n)] = cov@np.random.randn(p, int(cs[i] * n)) + mu_c1
            X_test[:,int(np.sum(cs[:i])*n_test):int(np.sum(cs[:i+1]*n_test))] =  cov@np.random.randn(p, int(cs[i] * n_test)) + mu_c1
                    
        # minority
        else:
            X[: , int(np.sum(cs[:i])*n):int(np.sum(cs[:i+1])*n)] =  np.sqrt(1 - beta**2) *  cov@np.random.randn(p, int(cs[i] * n)) + beta * mu_c1
            X_test[:,int(np.sum(cs[:i])*n_test):int(np.sum(cs[:i+1]*n_test))] = np.sqrt(1 - beta**2) *  cov@np.random.randn(p, int(cs[i] * n_test)) + beta * mu_c1
            
            # class 2
    else:
                # majority
        if i == 2:
            X[: , int(np.sum(cs[:i])*n):int(np.sum(cs[:i+1])*n)] =   cov@np.random.randn(p, int(cs[i] * n)) + mu_c2
            X_test[:,int(np.sum(cs[:i])*n_test):int(np.sum(cs[:i+1]*n_test))] =   cov@np.random.randn(p, int(cs[i] * n_test)) + mu_c2
                # minority
        else:
            X[: , int(np.sum(cs[:i])*n):int(np.sum(cs[:i+1])*n)] = np.sqrt(1 - beta**2) *  cov@np.random.randn(p, int(cs[i] * n)) + beta * mu_c2
            X_test[:,int(np.sum(cs[:i])*n_test):int(np.sum(cs[:i+1]*n_test))] = np.sqrt(1 - beta**2) * cov@np.random.randn(p, int(cs[i] * n_test)) + beta * mu_c2 
"""
for i in range(k):
    # class 1
    if i < 2:
        # majority
        if i == 0:
        #print(mu_c1.shape)
            X[: , int(np.sum(cs[:i])*n):int(np.sum(cs[:i+1])*n)] = cov@np.random.randn(p, int(cs[i] * n)) + mu_c1
            X_test[:,int(np.sum(cs[:i])*n_test):int(np.sum(cs[:i+1]*n_test))] =  cov@np.random.randn(p, int(cs[i] * n_test)) + mu_c1
                    
        # minority
        else:
            X[: , int(np.sum(cs[:i])*n):int(np.sum(cs[:i+1])*n)] =    cov@np.random.randn(p, int(cs[i] * n)) + beta * mu_c1
            X_test[:,int(np.sum(cs[:i])*n_test):int(np.sum(cs[:i+1]*n_test))] =  cov@np.random.randn(p, int(cs[i] * n_test)) + beta * mu_c1
            
            # class 2
    else:
                # majority
        if i == 2:
            X[: , int(np.sum(cs[:i])*n):int(np.sum(cs[:i+1])*n)] =   cov@np.random.randn(p, int(cs[i] * n)) + mu_c2
            X_test[:,int(np.sum(cs[:i])*n_test):int(np.sum(cs[:i+1]*n_test))] =   cov@np.random.randn(p, int(cs[i] * n_test)) + mu_c2
                # minority
        else:
            X[: , int(np.sum(cs[:i])*n):int(np.sum(cs[:i+1])*n)] = cov@np.random.randn(p, int(cs[i] * n)) + beta * mu_c2
            X_test[:,int(np.sum(cs[:i])*n_test):int(np.sum(cs[:i+1]*n_test))] = cov@np.random.randn(p, int(cs[i] * n_test)) + beta * mu_c2 
            
            
# kernel matrix
XX = X.T @ X / p
K = f(-2*XX+np.diag(XX).reshape(1,n)+np.diag(XX).reshape(n,1))
        
# Q & Q^{-1}
inv_Q = K + n / gamma * np.eye(n)
Q_y = np.linalg.solve(inv_Q, y)
Q_1 = np.linalg.solve(inv_Q, np.ones(n))
        
# alpha & b
b = np.sum(Q_y) / np.sum(Q_1)
alpha = Q_y - Q_1 * b
        
# soft score decision function 
g = lambda Y : alpha@f(np.diag(XX).reshape( (n, 1) ) + np.diag(Y.T@Y / p).reshape( (1,np.size(Y, 1)) ) - 2 * (X.T@Y / p)) + b
        
# compute score tests
g_test = g(X_test)


# theory
def fprime(x):
    return - 1 / 2 * f(x)

def fseconde(x):
    return - 1 / 2 * fprime(x)




In [9]:
for iter, xi in enumerate(xi_loop):
    store_output = [0 for _ in range(8)]
        #X = np.zeros( (p,n) )
        #X_test = np.zeros( (p, n_test) )
        
        #print(g_test)
        
    FAR_m = float(np.sum(g_test[  int(cs[0] * n_test) :  int(sum(cs[:2]) * n_test)] > xi)) / float(ceil(cs[1] * n_test))
    CDR_m = float( np.sum( g_test[ int((sum(cs[: - 1]) * n_test)) : ] > xi)) / float(ceil(cs[3] * n_test))
        
    FAR_M = float( np.sum(  g_test[  : int(cs[0] * n_test)  ] > xi)) /  float(ceil(cs[0] * n_test))
    CDR_M = float( np.sum( g_test[ int(sum(cs[:2]) * n_test) : int(sum(cs[:3]) * n_test)] > xi)) / float(ceil(cs[2] * n_test))
    
    FAR_theory0 = float( np.sum(g_theory[  : int(cs[0] * n_test) ] > xi)) / float( ceil(cs[0] * n_test) )
    CDR_theory0 = float( np.sum(g_theory[ int(sum(cs[:2]) * n_test) : int(sum(cs[:3]) * n_test) ] > xi)) / float( ceil(cs[2] * n_test) )
    
    FAR_theory1 = float( np.sum(g_theory[ int(cs[0] * n_test) :  int(sum(cs[:2]) * n_test) ] > xi)) / float ( ceil(cs[1] * n_test) )
    CDR_theory1 = float( np.sum(g_theory[ int((sum(cs[: - 1]) * n_test)) : ] > xi)) / float( ceil(cs[3] * n_test) )
    
    store_output[0] = FAR_m
    store_output[1] = CDR_m
    store_output[2] = FAR_M
    store_output[3] = CDR_M
    store_output[4] = FAR_theory0
    store_output[5] = CDR_theory0
    store_output[6] = FAR_theory1
    store_output[7] = CDR_theory1


    #print(store_output)
    store_error[iter, :] = store_output

#print(store_error[:,4])
#print(store_error[:,5])

def find_xi(labs, lord):
  imin = 0
  d2min = labs[0]**2 + (lord[0] - 1)**2
  for i in range(1, len(labs)):
    dist2 = labs[i]**2 + (lord[1] - 1)**2
    if dist2 < d2min:
      imin = i
      d2min = dist2
  return imin

# get indices of empirical xi
iminority = find_xi(store_error[ : , 0], store_error[ : , 1])
imajority = find_xi(store_error[ : , 2], store_error[ : , 3])
i0 = find_xi(store_error[ : , 4], store_error[ : , 5])
i1 = find_xi(store_error[ : , 6], store_error[ : , 7])
xi_theoric = (cs[2]+cs[3]) - (cs[1] + cs[0])
print("xi theorique = ", xi_theoric )
print("xi minorite = ", xi_loop[iminority])
print("xi majorite = ", xi_loop[imajority])
print("xi th2 0 = ", xi_loop[i0])
print("xi th2 1 = ", xi_loop[i1])
        
        
        

#print(store_error)
# DISPLAY ROC CURVE 
#plt.errorbar(xi_loop, store_error[:,0], store_error[:,1])
#plt.axvline(cs[1]-cs[0],ls='--',color='m')
#plt.xlabel(r'Decision threshold $\xi$')
#plt.ylabel('Misclassification rate')
%matplotlib qt
plt.plot(store_error[ : , 0 ], store_error[ : , 1 ], 'b', linestyle = '-')
plt.plot(store_error[ : , 2 ], store_error[ : , 3 ], 'red', linestyle = '-')
#plt.plot(store_error[ : , 4 ], store_error[ : , 5 ], 'orange', linestyle = '-')
#plt.plot(store_error[ : , 6 ], store_error[ : , 7 ], 'cyan', linestyle = '-')
plt.legend(["pratique, A = 1", "pratique, A = 0", "theoreme 2, A = 0", "theoreme 2, A = 1"])
#plt.legend(["pratique A = 0", "Theorie A = 0"])
plt.xlabel('False Alarm Rate')
plt.ylabel('Correct Decision Rate')
plt.show()        

xi theorique =  -0.125
xi minorite =  -0.12479435304579814
xi majorite =  -0.12479435304579814
xi th2 0 =  -0.1247610048910627
xi th2 1 =  -0.12478323699421966
